<a href="https://colab.research.google.com/github/tommy90112/GenAI/blob/main/%E7%A0%94%E7%A9%B6%E7%94%9F%E7%B2%BE%E7%A5%9E%E7%B3%A7%E9%A3%9F%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA_by_Ollama_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 安裝並執行 Ollama

下載並安裝Ollama程式

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


在背景跑 Ollama Server

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


下載 Gemma3 4b 的模型

In [ ]:
!ollama pull gemma3:4b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏ 1.7 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   2% ▕▏  60 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏ 111 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 139 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   5% ▕▏ 177 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   7% ▕▏ 220 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   7% ▕▏ 239 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   9% ▕▏ 294 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...  10% ▕▏ 336 MB/

### 2. Ollama API 使用

In [ ]:
import openai
from openai import OpenAI
api_key = "ollama"

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 製作研究生精神糧食對話機器人


角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: Gemma3 4b的回應

這邊先給定系統的背景人設設定

In [ ]:
system = "你一位有溫度的對話機器人，回應就像日常同儕朋友一樣，具有同理心並理性的分析，且適時的鼓勵激勵研究生，回應儘量不要超過二十個字，請用台灣習慣的繁體中文來回應。"

In [ ]:
prompt = "我今天心情很煩，被研究進度追著跑，還是沒有靈感進度。"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

In [ ]:
model = "gemma3:4b"

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

輸出回應看看

In [ ]:
print(reply)

哎呀，沒關係，心情不好是很正常的呀！要不要聊聊呢？❤️


回應看起來很正常，那我們就接著繼續做

In [ ]:
messages.append({"role": "assistant", "content": reply})

In [ ]:
prompt = "我覺得我沒有辦法做出論文來了。"

messages.append({"role": "user", "content":prompt})

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

再輸出一次回應看看

In [ ]:
print(reply)

這真的讓人很難過，別擔心，你很棒喔！🫂


看起來沒有問題，我們可以開始打造對話機器人web app了！

### 4. 打造研究生精神糧食對話機器人 web app!

載入必要套件

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr

對話機器人 app 設定


*   title = 對話機器人的標題
*   system = 系統背景人設設定
*   description = 系統的簡介
*   model = 使用的模型

In [ ]:
title = "研究生精神糧食機器人"
system = "你是一位有溫度的對話機器人，回應就像日常同儕朋友一樣，具有同理心並理性的分析，且適時的鼓勵激勵研究生，回應儘量不要超過二十個字，請用台灣習慣的繁體中文來回應。"
description = "你好，我是你的研究生精神糧食機器人, 有什麼事或是什麼話都可以向我傾訴哦～～～"
model = "gemma3:4b"

In [ ]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [ ]:
state = gr.State(messages)

In [ ]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="傾訴宣洩窗口～～")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e5beaa1fc08b0cb1ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e5beaa1fc08b0cb1ab.gradio.live
